In [1]:
!rm -rf tmp # cleaning up is necessary because the decoder will skip already existing files
!mkdir -p tmp
%cd tmp

/lustre/scratch/users/Jiayou.Zhang/genbio/ModelGenerator/experiments/AIDO.StructureTokenizer/tmp


/home/jiayou.zhang/miniconda3/envs/mgen/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# Get the sequence for structure prediction
from modelgenerator.structure_tokenizer.datasets.protein import Protein
from modelgenerator.structure_tokenizer.utils.constants import residue_constants as RC

# 6vxx A chain
pdb_id, chain_id = '6vxx', 'A'
!wget -qnc https://files.rcsb.org/download/{pdb_id}.pdb

aatype_tensor = Protein.from_pdb_file_path(f'{pdb_id}.pdb', chain_id).aatype
seq = "".join(list(RC.restype_1to3)[i] for i in aatype_tensor)
seq

'AYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVNCTFEYVSFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKGNYNYLYRKPFERDIYFPLQSYGFQPTNVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTSQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDS'

In [3]:
# dump the sequence to a csv file
import pandas as pd

df = pd.DataFrame(data={"idx": [0], "aa_seq": [seq], "seq_len": [len(seq)]})
df.to_csv("tmp.csv", index=False)

In [4]:
# language model: amino acid sequence -> structure tokens
!WANDB_MODE=dryrun mgen predict --config ../protein2structoken_16b.yaml \
            --data.init_args.path "csv" \
            --data.init_args.test_split_files ["tmp.csv"]

Seed set to 42
/home/jiayou.zhang/miniconda3/envs/mgen/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/jiayou.zhang/miniconda3/envs/mgen/bin/mgen pre ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
------------------------------------------------------------------------------------------------

In [5]:
# post process
!python ../struct_token_format_conversion.py logs/protein2structoken_16b/predict_predictions.tsv logs/protein2structoken_16b/predict_predictions.pt
!python ../extract_structure_tokenizer_codebook.py --output_path logs/protein2structoken_16b/codebook.pt

In [6]:
# Decode: structure tokens -> 3D coordinates
!WANDB_MODE=dryrun CUDA_VISIBLE_DEVICES=0 mgen predict --config ../decode.yaml \
 --data.init_args.config.struct_tokens_datasets_configs.name=protein2structoken_16b \
 --data.init_args.config.struct_tokens_datasets_configs.struct_tokens_path=./logs/protein2structoken_16b/predict_predictions.pt \
 --data.init_args.config.struct_tokens_datasets_configs.codebook_path=./logs/protein2structoken_16b/codebook.pt

/home/jiayou.zhang/miniconda3/envs/mgen/lib/python3.10/site-packages/lightning/fabric/utilities/seed.py:42: No seed found, seed set to 0
Seed set to 0
/home/jiayou.zhang/miniconda3/envs/mgen/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/jiayou.zhang/miniconda3/envs/mgen/bin/mgen pre ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 

In [7]:
# install visualization tool
!pip install py3Dmol -q
import py3Dmol

In [ ]:
# visualize the prediction and the ground truth
def visualize(file):
    view = py3Dmol.view(query='pdb')
    with open(file, 'r') as f:
        view.addModel(f.read(), 'pdb')
    view.setStyle({'cartoon': {'color': 'spectrum'}})
    view.zoomTo()
    view.show()

prediction = "logs/protstruct_decode/protein2structoken_16b_pdb_files/0__output.pdb"
ground_truth = f"{pdb_id}_{chain_id}.pdb"
# drop the additional chain in the ground truth before visualization
Protein.from_pdb_file_path(f'{pdb_id}.pdb', chain_id).to_pdb(f"{pdb_id}_{chain_id}.pdb")

print("Ground truth:")
visualize(f"{pdb_id}_{chain_id}.pdb")
print("Prediction:")
visualize(prediction)

Ground truth:


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Prediction:


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.